In [1]:
import pandas as pd
import pydicom
import numpy as np
import os
import matplotlib.pylab as plt
import seaborn as sns
from skimage import exposure
from skimage import io
import cv2 as cv
import sys

In [2]:
def resize(image, image_size = 256):
    
    image = cv.resize(image,(image_size,image_size))
    return image

def to_int(image):

    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image,0) / image.max()) * 255.0

    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)
    
    return image_2d_scaled

In [6]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):
    
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

def bsb_window(dcm):
    
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 600, 2800)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    
    
    brain_img = exposure.equalize_adapthist(brain_img)
    brain_img = to_int(brain_img)
    brain_img = resize(brain_img)
    
    subdural_img = exposure.equalize_adapthist(subdural_img)
    subdural_img = to_int(subdural_img)
    subdural_img = resize(subdural_img)
    
    soft_img = to_int(soft_img)
    soft_img = resize(soft_img)
    
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

    return bsb_img

In [7]:
path = 'stage_1_train_images.dcm'

image_set = os.listdir(path)
len(image_set)

674259

In [9]:
df = pd.read_csv('RSNA.csv')
df.head()

,ID,Any,Epidural,Intraparenchymal,Intraventricular,Subarachnoid,Subdural
0,ID_000039fa0,0,0,0,0,0,0
1,ID_00005679d,0,0,0,0,0,0
2,ID_00008ce3c,0,0,0,0,0,0
3,ID_0000950d7,0,0,0,0,0,0
4,ID_0000aee4b,0,0,0,0,0,0


In [10]:
df = df.drop(df[df.Any == 0].index)
df.head()

,ID,Any,Epidural,Intraparenchymal,Intraventricular,Subarachnoid,Subdural
12,ID_0002081b6,1,0,1,0,0,0
21,ID_0002a38ad,1,0,0,0,1,1
28,ID_000346ce2,1,0,0,0,0,1
31,ID_00042829c,1,0,0,1,0,0
37,ID_0004a5701,1,0,0,0,0,1


In [ ]:
for i,item in df.iterrows():
    
    image = pydicom.dcmread(path +'/'+ item[0]+'.dcm')
    image = bsb_window(image)
    io.imsave('ICH_dcm/'+ filename[:-4] + '.dcm', image)

In [10]:
for filename in image_set[264485:]:
    
    try:
        
        image = pydicom.dcmread(path +'/'+ filename)
        image = bsb_window(image)
        io.imsave('ICH_png/'+ filename[:-4] + '.png', image)
        
    except ValueError:
        
        pass

C:\Users\asarkar.NATICK-NT\.conda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: ICH_png/ID_6431f0645.png is a low contrast image
  import sys
C:\Users\asarkar.NATICK-NT\.conda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: ICH_png/ID_6433dcc46.png is a low contrast image
  import sys
C:\Users\asarkar.NATICK-NT\.conda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: ICH_png/ID_643932954.png is a low contrast image
  import sys
C:\Users\asarkar.NATICK-NT\.conda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: ICH_png/ID_643a45caa.png is a low contrast image
  import sys
C:\Users\asarkar.NATICK-NT\.conda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: ICH_png/ID_643cef2d8.png is a low contrast image
  import sys
C:\Users\asarkar.NATICK-NT\.conda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: ICH_png/ID_6448aeab8.png is a low contrast image
  import sys
C:\Users\asarkar

InvalidDicomError: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.